# Using the NengoEdge Micro Runner

In this example we will walk through loading and running a model exported from
NengoEdge
that's been uniquely configured to run on micro devices supporting TFLite Micro
(we support the STM32F746 Discovery Board (Disco) and nRF52840 Dev Board
(Nordic)). The goal
of this demo is to provide a template for you to make your own custom
applications using
the NengoEdge runner.

## Installation

First, install NengoEdge tools using
[these instructions](https://www.nengo.ai/nengo-edge/developers.html).
Feel free to skip the Tensorflow step since we won't need it for this
runner.

For the micro device example, it's assumed that you have completed the
installation steps for either your Disco or Nordic board,

- [Nordic guide
](https://www.nordicsemi.com/Products/Development-software/nrf-connect-sdk)
- [Disco guide
](https://wiki.st.com/stm32mcu/wiki/Microcontroller)

Take note of the **serial** and **drive** paths that are associated with your
micro device as these are **required** for the Python runner. For example,
the STM Discovery Board should generate serial and drive paths on your
system named something like `/dev/ttyACM0`
and `/media/<username>/DIS_F746NG` respectively (exact names will depend on
your system).

## Train a model in NengoEdge

The first step is to train a model in NengoEdge for your desired hardware. See
[this blog post
](https://appliedbrainresearch.com/blog/fast-keyword-detection-with-lmus-on-gpu)
for a detailed walkthrough on how to train such a model.

## Export the trained model

When exporting the model from NengoEdge you must choose the "BINARY" option to get a
model targeted to run on microcontroller devices that use TFLiteMicro. The downloaded
artifacts can be unpacked to a directory of your choice, and for the purpose of this
demo we'll assume the contents have been unpacked to a directory called `micro_demo/`.

Inside this directory you'll find two files (note: the binary file extension may change
depending on the specific device):

- `nengoedge_project.bin`
- `parameters.json`

We'll create a `DiscoRunner` that utilizes these artifacts.

Note that if you are running this code locally, you will need to uncomment the
`os.environ` lines and update `<DISCO_SERIAL_PATH>`/`<DISCO_DRIVE_PATH>` to point to
the serial/drive path (see the Installation steps above).

In [1]:
import os
import numpy as np

from nengo_edge import DiscoRunner

# os.environ["DISCO_SERIAL_PATH"] = <DISCO_SERIAL_PATH>
# os.environ["DISCO_DRIVE_PATH"] = <DISCO_DRIVE_PATH>
runner = DiscoRunner(
    directory="micro_demo",
    serial_path=os.environ["DISCO_SERIAL_PATH"],
    device_path=os.environ["DISCO_DRIVE_PATH"],
)

2024-04-29 18:03:36.347570: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-29 18:03:36.369605: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-29 18:03:36.370044: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-04-29 18:03:36.786650: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Output()

/runner/_work/nengo-edge/nengo-edge/nengo-edge/nengo_edge/config.py:37: UserWarning: Downloaded model uses nengo-edge 23.9.11.dev0, but you're using nengo-edge 24.3.6.dev0. Mismatch may cause errors.
  warnings.warn(


You will likely have particular audio inputs you are interested in identifying with the
edge key word spotting model, but we'll showcase the general run steps assuming a random
signal.

In [2]:
audio_inputs = np.random.uniform(
    -1, 1, (1, runner.preprocessing["sample_rate"])
).astype("float32")

# Keyword labels for model outputs
labels = [
    "<silence>",
    "<unknown>",
    "yes",
    "no",
    "up",
    "down",
    "left",
    "right",
    "on",
    "off",
    "stop",
    "go",
]

And finally we can run the model using a microcontroller device,

In [3]:
# using the runner's context opens the serial communication
# to the board set by serial_path
with runner:
    outputs = runner.run(audio_inputs)
    pred_label = np.argmax(outputs)
    print(f"Predicted keyword: {labels[pred_label]}")

Output()

Predicted keyword: off


Since we used a random audio sample here and the model was trained on real audio
samples, it's likely this particular classification will
result in a random label. You'll notice that in this example we only ran the model
on a single batch of inputs but the runner also supports batched inputs.

That's it! You're now set to take this runner into your own applications.